In [ ]:
import os
import gc
import time
import pickle
from pathlib import Path

import numpy as np
np.random.seed(0)

# import lightgbm as lgb
from tqdm import tqdm

import mxnet as mx
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
mx.random.seed(0)

from train_models import *
from train_utils import *
from utils import *

import warnings

warnings.filterwarnings("ignore", message="WARNING:gluonts.model.forecast:The mean prediction is not stored in the forecast data; the median is being returned instead. This behaviour may change in the future.")

In [ ]:
def train_estimator(epochs, learning_rate, estimator_name):    
    save_dir = f'../result'
    os.makedirs(save_dir, exist_ok=True)
    
    for level_idx in range(1, 13):
        level_dir = os.path.join(save_dir, f'level {level_idx}')
        os.makedirs(level_dir, exist_ok=True)
        estimator_dir = os.path.join(level_dir, estimator_name)
        if any(existing_dir.startswith(estimator_name) for existing_dir in os.listdir(level_dir)):
            continue
        os.makedirs(estimator_dir, exist_ok=True)

        highlight_print(f"\n========== Level {level_idx} ==========")
        print(f"Loading dataset...", end=' ')
        dataset_start = time.time()
        with open(os.path.join('../dataset/else', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
            dataset = pickle.load(f)
        print(f"{(time.time() - dataset_start)/60:.1f} minutes")

        estimator = create_estimator(
            level_idx=level_idx, 
            train_dataset=dataset['train'],
            estimator_name=estimator_name,
        )  

        estimator_start = time.time()
        print("Start training...")
        train_start = time.time()
        estimator.trainer = EarlyStoppingTrainer(
            epochs=epochs,
            learning_rate=learning_rate,
            num_batches_per_epoch=get_optimal_num_batches(mx.context.num_gpus()),
            patience=15,
        )
        predictor = estimator.train(
            training_data=dataset['train'],
            validation_data=dataset['valid']
        )
        predictor.serialize(Path(f"{level_dir}/{estimator_name}"))
        highlight_print(f"End training... {(time.time() - train_start)/60:.1f} minutes", color='green')

        print("Start predicting...")
        pred_start = time.time()
        test_forecasts_it, test_labels_it = make_evaluation_predictions(
            dataset=dataset['test'],
            predictor=predictor,
        )
        test_forecasts = list(test_forecasts_it)
        test_labels = list(test_labels_it)
        highlight_print(f"End predicting... {(time.time() - pred_start)/60:.1f} minutes", color='green')

        print("Start saving...")
        pred_save_start = time.time()
        with open(f"{level_dir}/{estimator_name}/test_labels.pkl", "wb") as f:
            pickle.dump(test_labels, f)
        with open(f"{level_dir}/{estimator_name}/test_forecasts.pkl", "wb") as f:
            pickle.dump(test_forecasts, f)
        highlight_print(f"End saving... {(time.time() - pred_save_start)/60:.1f} minutes", color='green')

        # print("Start plotting...")
        # plot_start = time.time()
        # for i in range(len(test_forecasts)):
        #     plt.figure(figsize=(12, 6))
        #     plt.plot(test_labels[i][-100:].to_timestamp(), label="Actual")
        #     plt.plot(pd.Series(test_forecasts[i].quantile(0.5), index=test_forecasts[i].start_date.to_timestamp() + pd.to_timedelta(range(len(test_forecasts[0].quantile(0.5))), unit='D')), label="Forecast")
        #     plt.title(f'{test_forecasts[i].item_id}')
        #     plt.xlabel('Date')
        #     plt.ylabel('Sales')
        #     plt.legend(loc="upper right")
        #     plt.savefig(os.path.join(estimator_dir, f'series_{i + 1}.png'))
        #     plt.close()
        # highlight_print(f"End plotting... {(time.time() - plot_start)/60:.1f} minutes", color='green')
            
        print("Start evaluating...")
        eval_start = time.time()
        evaluator = Evaluator(quantiles=(0.5,), ignore_invalid_values=True)
        test_metrics_all_id, test_metrics_per_id = evaluator(test_labels, test_forecasts)
        highlight_print(f"End evaluating... {(time.time() - eval_start)/60:.1f} minutes", color='green')

        print("Start saving...")
        eval_save_start = time.time()
        with open(f"{level_dir}/{estimator_name}/test_metrics_all_id.pkl", "wb") as f:
            pickle.dump(test_metrics_all_id, f)
        with open(f"{level_dir}/{estimator_name}/test_metrics_per_id.pkl", "wb") as f:
            pickle.dump(test_metrics_per_id, f)
        highlight_print(f"End saving... {(time.time() - eval_save_start)/60:.1f} minutes", color='green')

        highlight_print(f"\nTotal time: {(time.time() - estimator_start)/60:.1f} minutes", color='red')

        # rename estimator directory
        os.rename(estimator_dir, f"{estimator_dir}_{test_metrics_all_id['MSE']:.2f}")

        # reduce memory
        del estimator, predictor, test_forecasts, test_labels
        gc.collect()

train_estimator(epochs=500, learning_rate=1e-3, estimator_name='WaveNet')


========== Level 1 ==========
Loading dataset... 0.0 minutes
Start training...


100%|██████████| 143/143 [00:22<00:00,  6.26it/s, epoch=1/500, avg_epoch_loss=4.74]
1it [00:00, 142.87it/s, epoch=1/500, validation_avg_epoch_loss=4.11]
100%|██████████| 143/143 [00:22<00:00,  6.33it/s, epoch=2/500, avg_epoch_loss=4.09]
1it [00:00, 125.00it/s, epoch=2/500, validation_avg_epoch_loss=4.71]
100%|██████████| 143/143 [00:22<00:00,  6.33it/s, epoch=3/500, avg_epoch_loss=3.45]
1it [00:00, 142.83it/s, epoch=3/500, validation_avg_epoch_loss=6.03]
100%|██████████| 143/143 [00:23<00:00,  6.21it/s, epoch=4/500, avg_epoch_loss=3.28]
1it [00:00, 142.87it/s, epoch=4/500, validation_avg_epoch_loss=6.56]
100%|██████████| 143/143 [00:22<00:00,  6.29it/s, epoch=5/500, avg_epoch_loss=3.14]
1it [00:00, 166.67it/s, epoch=5/500, validation_avg_epoch_loss=7.07]
100%|██████████| 143/143 [00:22<00:00,  6.30it/s, epoch=6/500, avg_epoch_loss=2.99]
1it [00:00, 166.71it/s, epoch=6/500, validation_avg_epoch_loss=7.97]
100%|██████████| 143/143 [00:22<00:00,  6.31it/s, epoch=7/500, avg_epoch_loss=2.82

Early stopping triggered
Restored best estimator
End training... 6.1 minutes
Start predicting...
End predicting... 0.0 minutes
Start saving...
End saving... 0.0 minutes
Start evaluating...


Running evaluation: 1it [00:00, 250.02it/s]


End evaluating... 0.0 minutes
Start saving...
End saving... 0.0 minutes

Total time: 6.1 minutes

========== Level 2 ==========
Loading dataset... 0.0 minutes
Start training...


...

100%|██████████| 119/119 [00:21<00:00,  5.58it/s, epoch=1/500, avg_epoch_loss=4.91]
96it [00:07, 12.29it/s, epoch=1/500, validation_avg_epoch_loss=5.34]
100%|██████████| 119/119 [00:20<00:00,  5.67it/s, epoch=2/500, avg_epoch_loss=4.17]
96it [00:07, 12.14it/s, epoch=2/500, validation_avg_epoch_loss=5.22]
100%|██████████| 119/119 [00:21<00:00,  5.62it/s, epoch=3/500, avg_epoch_loss=3.98]
96it [00:07, 12.09it/s, epoch=3/500, validation_avg_epoch_loss=5.15]
100%|██████████| 119/119 [00:21<00:00,  5.53it/s, epoch=4/500, avg_epoch_loss=3.96]
96it [00:08, 11.84it/s, epoch=4/500, validation_avg_epoch_loss=5.09]
100%|██████████| 119/119 [00:21<00:00,  5.58it/s, epoch=5/500, avg_epoch_loss=3.92]
96it [00:07, 12.10it/s, epoch=5/500, validation_avg_epoch_loss=5.05]
100%|██████████| 119/119 [00:22<00:00,  5.27it/s, epoch=6/500, avg_epoch_loss=3.96]
96it [00:08, 11.17it/s, epoch=6/500, validation_avg_epoch_loss=5.02]
100%|██████████| 119/119 [00:21<00:00,  5.52it/s, epoch=7/500, avg_epoch_loss=3.85

End training... 242.8 minutes
Start predicting...
End predicting... 23.4 minutes
Start saving...
End saving... 0.0 minutes
Start evaluating...


Running evaluation: 3049it [00:11, 270.71it/s]
c:\Users\USER\anaconda3\envs\sci2\lib\site-packages\pandas\core\dtypes\astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


End evaluating... 0.2 minutes
Start saving...
End saving... 0.0 minutes

Total time: 266.4 minutes

========== Level 11 ==========
Loading dataset... 0.3 minutes
Start training...


100%|██████████| 94/94 [00:19<00:00,  4.86it/s, epoch=1/500, avg_epoch_loss=4.75]
286it [00:21, 13.02it/s, epoch=1/500, validation_avg_epoch_loss=4.79]
100%|██████████| 94/94 [00:17<00:00,  5.49it/s, epoch=2/500, avg_epoch_loss=3.41]
286it [00:22, 12.97it/s, epoch=2/500, validation_avg_epoch_loss=4.62]
100%|██████████| 94/94 [00:16<00:00,  5.58it/s, epoch=3/500, avg_epoch_loss=3.24]
0it [00:00, ?it/s]

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='MQCNN')

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='MQRNN')

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='NBEATS')

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='DeepState')

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='Transformer')

In [ ]:
train_estimator(epochs=500, learning_rate=1e-3, estimator_name='DeepFactor')